In [ ]:
# STEP 1: Install all the necessary libraries.
# LangChain for orchestration, LangChain-Groq for Groq LLM.
!pip install -q langchain langchain-groq langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [ ]:
# STEP 2: Gather your spellbooks.
from langchain_groq import ChatGroq
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)
from langchain_core.output_parsers.xml import XMLOutputParser  # 🌟 The XML parser hero!
from IPython.display import Markdown, display
from google.colab import userdata
import os

In [ ]:
# STEP 3: Whisper your secret key to the winds.
try:
    api_key = userdata.get("GROQ_API_KEY")
except Exception:
    api_key = os.getenv("GROQ_API_KEY")
    if not api_key:
        raise ValueError("GROQ_API_KEY not found. Please set it!")

# STEP 3b: Summon the Groq-powered LLM.
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=api_key,
    temperature=0.3,
)

In [ ]:
# STEP 4: 📜 Instruct your LLM to always output valid XML.
# -----------------------------------------------
# ⚡️ CRUCIAL: If you do not explicitly prompt your LLM to respond in valid XML,
# XMLOutputParser will fail because it expects well-formed XML!
# -----------------------------------------------

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an XML generator assistant. Always respond in well-formed XML with <response> root tag."
)

human_msg = HumanMessagePromptTemplate.from_template(
    "{input}"
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_msg,
    MessagesPlaceholder(variable_name="history"),
    human_msg
])

In [ ]:
# STEP 5: Archive the echoes of your conversations.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [ ]:
# STEP 6: 🗝️ The heart of this spell!
# -----------------------------------------------
# 💡 XMLOutputParser expects valid XML and parses it into ElementTree.
# It will raise an error if the LLM’s output is not valid XML.
# So your prompt must always guide the LLM carefully.
# -----------------------------------------------

chat_chain = chat_prompt | llm

# Add the XML parser
parsed_chain = chat_chain | XMLOutputParser()

# Wrap with memory so your assistant remembers the scrolls.
chatbot = RunnableWithMessageHistory(
    parsed_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [ ]:
import xml.etree.ElementTree as ET

session_id = "chat-session-xml-001"

user_inputs = [
    "Describe the confusion matrix in an XML format with fields <answer> and <source>.",
    "Give an example of precision and recall wrapped in <answer> and <source>."
]

print(f"Starting chat loop with session ID: {session_id}")

for input_text in user_inputs:
    print(f"\nUser: {input_text}")
    response_text = chatbot.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    print("Raw Response:", response_text)

    response_tree = ET.fromstring(response_text)
    answer = response_tree.find("answer").text if response_tree.find("answer") is not None else "No <answer> found"
    source = response_tree.find("source").text if response_tree.find("source") is not None else "No <source> found"
    display(Markdown(f"**Answer:** {answer}\n\n**Source:** {source}"))

print("\n--- Stored Chat History ---")
for message in store[session_id].messages:
    print(f"{message.type.capitalize()}: {message.content}")


Starting chat loop with session ID: chat-session-xml-001

User: Describe the confusion matrix in an XML format with fields <answer> and <source>.


ValueError: Message dict must contain 'role' and 'content' keys, got {'answer': 'The confusion matrix is a table used to evaluate the performance of a classification model. It has four main components: true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN). The matrix is used to calculate various metrics such as accuracy, precision, recall, and F1 score.'}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 